# StableSwap Rust (USDC/USDT, 6 decimals)

> **Abstract.** This crate provides a pure-math implementation of Curve's StableSwap algorithm for a two-asset stablecoin pool (USDC/USDT, 6 decimals). We formalize the invariant, derive the Newton iterations for the pool invariant $D$ and for the post-trade balance $y$, and discuss integer-precision arithmetic, rounding policy, numerical stability, and complexity. A constant-product baseline is also included for slippage comparison. The code is entirely deterministic and uses only integer arithmetic (`u128`), which makes it suitable for on-chain or reproducible off-chain contexts.

---

# 1. StableSwap Rust — Install via Git & Usage Guide

> **TL;DR**
>
> * Package name (Cargo dependency key): `stable-swap-rs`
> * **Crate name** (what you `use` in Rust): `stable_swap_rs`
> * Token amounts use **6 decimals** (e.g., `1 USDC = 1_000_000`)

---

### 1.1. Overview

A pure-math implementation of Curve’s **StableSwap** algorithm for a **two-asset** stablecoin pool (USDC/USDT).

* Invariant **D** and post-trade **y** via Newton’s method
* Integer-only arithmetic (`u128` internally)
* Input-side fee in **bps**
* Constant-product (x\*y=k) baseline for slippage comparison

This library is chain-agnostic and suitable for on-chain ports or off-chain simulation.

---

### 1.2. Requirements

* **Rust** 1.67+ (2021 edition)
* Platform with `u128` integer support (stable Rust toolchains do)

---

### 1.3. Install from Git (Recommended)

Add the dependency to your project’s `Cargo.toml` (replace with your real GitHub URL and an immutable tag or commit):

```toml
[dependencies]
# Git tag (recommended)
stable-swap-rs = { git = "https://github.com/zeron-G/Rust-Developer-Test-Assignment_StableSwap-Implementation", tag = "v0.1.0" }
# Or pin to a commit
# stable-swap-rs = { git = "https://github.com/zeron-G/Rust-Developer-Test-Assignment_StableSwap-Implementation", rev = "<commit-hash>" }
```

> **Note on names**
> The **package name** is `stable-swap-rs` (used in `Cargo.toml`).
> The **crate name** is `stable_swap_rs` (used in `use` statements).

---

### 1.4. Quickstart

Create a small binary to try the library:

```bash
cargo new demo-stable --bin
cd demo-stable
```

Edit `demo-stable/Cargo.toml` and add the Git dependency as shown above. Then paste the following into `src/main.rs`:

```rust
use stable_swap_rs::{StableSwapPool, constant_product_dy};

fn main() -> anyhow::Result<()> {
    // Amounts are 6-decimal integers: 1 USDC = 1_000_000
    // Build a balanced pool: 10,000 USDC & 10,000 USDT
    let mut pool = StableSwapPool::new(
        [10_000_000_000, 10_000_000_000], // 10,000 * 1e6
        100, // A = 100
    );
    pool.fee_bps = 30; // 0.30% input-side fee

    // Quote: swap 100 USDC -> USDT
    let dx: u64 = 100_000_000; // 100 * 1e6
    let dy = pool.get_dy(0, 1, dx).expect("quote failed");
    println!("StableSwap quote: 100 USDC -> {} USDT (6dp int)", dy);

    // Compare with constant product (no fee)
    let dy_xyk = constant_product_dy(pool.reserves, 0, 1, dx).unwrap();
    println!("x*y=k (no fee) output: {}", dy_xyk);

    // --- Executing the swap (caller updates reserves) ---
    let fee_bps = pool.fee_bps as u128;
    let dx_net = (dx as u128) * (10_000u128 - fee_bps) / 10_000u128;

    pool.reserves[0] = (pool.reserves[0] as u128 + dx_net) as u64; // add net input
    pool.reserves[1] = pool.reserves[1].checked_sub(dy)
        .expect("insufficient reserve for execution"); // subtract output

    // Optional: inspect invariant D after trade
    let d_now = pool.get_d();
    println!("D after trade: {}", d_now);

    // Curve-only slippage estimate (ignoring fees)
    let s_bps = pool.calculate_slippage_bps(100_000_000);
    println!("curve-only slippage for 100 USDC: {} bps", s_bps);

    Ok(())
}
```

Run it:

```bash
cargo run -q
```

---

### 1.5. API Snapshot

```rust
pub struct StableSwapPool {
    pub reserves: [u64; 2],
    pub amplification_coefficient: u64, // A
    pub fee_bps: u16,                   // input-side fee in bps
}
impl StableSwapPool {
    pub fn new(reserves: [u64; 2], amp: u64) -> Self;
    pub fn get_dy(&self, i: usize, j: usize, dx: u64) -> Result<u64, SwapError>;
    pub fn get_d(&self) -> u64;
    pub fn calculate_slippage_bps(&self, amount: u64) -> u16;
}

/// Constant-product baseline (no fee)
pub fn constant_product_dy(reserves: [u64;2], i: usize, j: usize, dx: u64) -> Option<u64>;
```

#### Units

* **All token amounts** are integers with **6 decimals** (USDC/USDT standard on Solana, but general-purpose here).
  Example: `1 USDC = 1_000_000`.

#### Errors

`get_dy` may return: `BadIndex`, `ZeroTrade`, `NoLiquidity`, `NoConvergence`, `Math`.
Callers should handle these explicitly.

---

### 1.6. Executing a Swap (State Update Contract)

`get_dy` is a **pure quote**. To "execute" a trade, the caller is responsible for mutating pool state consistently:

1. Compute net input: `dx_net = dx * (10000 - fee_bps) / 10000`
2. Update reserves:

   * `reserves[i] += dx_net`
   * `reserves[j] -= dy`
3. (Optional) Recompute `D` for observability/metrics

This separation allows you to quote multiple trades safely before committing.

---

### 1.7. Design Notes (What To Expect)

* **Integer-only math**: internal arithmetic uses `u128`; fractions are evaluated via a safe `mul_div` (checked multiply then divide).
* **Conservative rounding**: outputs are rounded down and reduced by 1 minimal unit to protect the invariant (standard on-chain practice).
* **Low slippage near balance**: with moderate `A` (e.g., 50–200) and balanced reserves, StableSwap outputs are strictly higher than constant-product for the same input.
* **Graceful degradation**: far from balance and for large trades, behavior transitions toward constant-product.

---

### 1.8. Versioning & Minimum Supported Rust Version (MSRV)

* **Versioning**: semantic versioning (`MAJOR.MINOR.PATCH`). Pin a **tag** or commit in your `Cargo.toml` for reproducibility.
* **MSRV**: tested with Rust 1.67+ (Edition 2021). We aim to avoid unstable features.

---

### 1.9. Testing

Clone the library and run:

```bash
cargo test -q
```

The tests cover: monotonicity of `D`, fee-on-input behavior, and lower slippage vs. x\*y=k in balanced pools.

---

## 1.10. Support & Contributions

* **Issues**: please open a GitHub issue with a minimal reproduction.
* **PRs**: welcome! Keep changes well-scoped, add tests, and run `cargo fmt`/`cargo clippy`.



## 2. theroy

### 2.1. Notation and Problem Statement

Let $n=2$ be the number of coins in the pool with balances $x_0, x_1>0$, and define

$$
S = \sum_{i=0}^{n-1} x_i, \qquad P = \prod_{i=0}^{n-1} x_i .
$$

Let $A>0$ denote the amplification coefficient and $Ann = A\,n^n$ (for two coins, $Ann=4A$). All quantities are represented as integers with 6 fraction digits (e.g., `1 USDC = 1_000_000`). Internal arithmetic is performed in $\mathtt{u128}$.

**Objective.** Given an input trade of size $\Delta x$ on coin $i$, we seek the output $\Delta y$ on coin $j\ne i$ such that the trade preserves the StableSwap invariant (up to conservative rounding) and applies a basis-points fee on the input.

---



### 2.2. StableSwap Invariant (Two-Asset Case)

The invariant in the two-asset case is written as

$$
Ann\,S + D \;=\; Ann\,D + \frac{D^{n+1}}{n^n\,P}\qquad (n=2),
$$

where $D$ is the positive solution balancing the curve. Define

$$
F(D) \equiv Ann\,S + D - Ann\,D - \frac{D^{n+1}}{n^n\,P} = 0 .
$$

We compute $D$ by Newton’s method. Let

$$
D_P(D) \equiv \frac{D^{n+1}}{n^n\,P}, \qquad 
F'(D) = 1 - Ann - \frac{(n+1)D^n}{n^n\,P}
     = 1 - Ann - \frac{(n+1)D_P}{D}.
$$

Then

$$
D_{\text{new}} 
= D - \frac{F(D)}{F'(D)}
= D - \frac{AnnS + D - AnnD - D_P}{1 - Ann - (n+1)D_P/D}.
$$

Multiplying numerator/denominator by $D$ yields the closed-form update used in code:

$$
\boxed{\; D_{\text{new}} = \frac{\big(AnnS + nD_P\big)\,D}
{(Ann-1)D + (n+1)D_P} \;}
$$

with $n=2$. Iterate until $|D_{k+1}-D_k|\le 1$ (or a hard cap). In practice, convergence occurs in <10 steps.

**Safe evaluation of $D_P$.** Rather than forming $D^{n+1}$ directly, build it iteratively to avoid overflow:

$$
D_P \leftarrow D,\quad 
D_P \leftarrow (D_P\cdot D)/(x_0\cdot n),\quad
D_P \leftarrow (D_P\cdot D)/(x_1\cdot n).
$$

All products/divisions use checked integer arithmetic (`mul_div`).

---

### 2.3. Solving for the Post-Trade Balance $y$

After charging the input fee (see §4), set $x_i \leftarrow x_i + \Delta x_{\text{net}}$. Solve for the new output-side balance $y\equiv x'_j$ **holding $D$ fixed**. Let

$$
S' = \sum_{k\ne j} x_k,\qquad Ann = A\,n^n .
$$

With $D$ fixed, the invariant reduces to a scalar nonlinear equation in $y$. A standard rearrangement yields

$$
f(y) \equiv y^2 + (b - D)\,y - c = 0,
$$

where

$$
b = S' + \frac{D}{Ann}, \qquad
c = \frac{D^{n+1}}{n^n\,\prod_{k\ne j} x_k}\cdot \frac{1}{Ann} .
$$

Applying Newton’s method to $f(y)$ gives the practical update:

$$
\boxed{\; y \leftarrow \frac{y^2 + c}{2y + b - D} \;}
$$

Initialize $y\gets D$ and iterate until $|y_{k+1}-y_k|\le 1$. The output amount is

$$
\Delta y = x_j - y - 1,
$$

where the extra `-1` is a conservative guard against invariant drift due to integer truncation.

---

### 2.4. Fees, Price, and Slippage

Let `fee_bps` be the input-side fee in basis points (1% = 100 bps). We charge the fee on the input:

$$
\Delta x_{\text{net}} = \Delta x\cdot \frac{\text{BPS}_\text{denom} -\text{feebps}}{\text{BPS}\text{denom}}
$$

$$
\qquad \text{BPS}_\text{denom}=10{,}000.
$$

The execution price is $p=\Delta x/\Delta y$. To measure **curve-only** slippage, set fee to zero temporarily and compute

$$
\text{slippagebps} \approx 10^4\,(p-1),
$$

relative to the ideal 1:1 price.

---

### 2.5. Integer Arithmetic, Precision, and Rounding

* **Representation.** Token amounts are 6-decimal fixed-point integers (`u64` externally). Internal math promotes to `u128`.\n- **Exact integer math.** No floating point. Fractions via `mul_div(a,b,den)` (checked multiply, then checked divide).\n- **Conservative rounding.** After solving $y$, round down and subtract one minimal unit (`-1`).\n- **Convergence.** Both Newton solvers stop when successive iterates differ by ≤1 or after a hard cap (`MAX_ITERS=256`). Typical iterations <10.

---

### 2.6. Constant-Product Baseline

For $x\,y=k$ (no fee), a trade of size $\Delta x$ yields

$$
y' = \frac{k}{x+\Delta x},\qquad \Delta y = y - y' - 1.
$$

We expose this as `constant_product_dy` for direct slippage comparison; StableSwap dominates near balance for reasonable $A$.

---

### 2.7. Complexity and Stability

* **Time complexity.** $\mathcal{O}(1)$ work per Newton step; tiny constant factors.\n- **Stability.** `mul_div` encapsulates multiply-then-divide; denominators checked; edge cases (`BadIndex`, `ZeroTrade`, `NoLiquidity`, `NoConvergence`, `Math`) return explicit errors.\n- **Behavior.** Near balance and with sizeable $A$, the curve approximates constant-sum (very low slippage); far from balance it transitions toward constant-product, ensuring finite liquidity.

## 2.8. Practical Guidance

* **Choosing $A$.** Larger $A$ flattens the curve near $x_0\!\approx\!x_1$ while retaining constant-product behavior when imbalanced.\n- **Do not cache $D$.** Always recompute from current reserves.\n- **Rounding policy.** The conservative “round down and `-1`” protects LPs; adjust tests if you move fees to the output side.

---

## 2.9. Reproducibility

The library is deterministic and integer-only, suitable for on-chain and off-chain verification.

---

## 3. Reference

* M. Egorov, *StableSwap: An efficient mechanism for Stablecoin liquidity*, 2019.


下面我把**两个牛顿迭代**的数学原理与全过程讲透：
(1) 求不变量 $D$ 的牛顿法；(2) 在保持 $D$ 不变时求输出侧余额 $y$ 的牛顿法。我会从白皮书的不变量出发，推导目标方程 $F=0$ 或 $f=0$、求导、化简出**闭式的牛顿步**，并解释为什么它们数值上稳定、为何取这样的初值、何时终止。

---

## 一、求不变量 $D$ 的牛顿法

### 1. 起点：StableSwap 不变量（两币情形）

对两币池（USDC/USDT，$n=2$）记：

* 余额 $x_0,x_1>0$
* 和 $S=x_0+x_1$，积 $P=x_0x_1$
* 放大系数 $A>0$，定义 $\mathrm{Ann}=A\cdot n^n=A\cdot 4$

白皮书给出不变量：

$$
\mathrm{Ann}\,S + D \;=\; \mathrm{Ann}\,D + \frac{D^{n+1}}{n^n\,P}
\quad(n=2).
$$

把它改写为求根问题 $F(D)=0$：

$$
F(D) \;\equiv\; \mathrm{Ann}\,S + D - \mathrm{Ann}\,D \;-\; \frac{D^{n+1}}{n^n\,P}.
$$

> 直观：当 $D$ 过小，左边 $\mathrm{Ann}\,S + D$ 偏大、右边 $\mathrm{Ann}\,D + \dots$ 偏小，$F(D)>0$；当 $D$ 很大时，$\tfrac{D^{n+1}}{n^nP}$ 主导，$F(D)\to -\infty$。因而**存在唯一正根**。

### 2. 牛顿法公式

牛顿法的通式是

$$
D_{\text{new}} \;=\; D - \frac{F(D)}{F'(D)}.
$$

我们先算导数 $F'(D)$。写

$$
D_P(D)\;\equiv\; \frac{D^{n+1}}{n^n\,P},
$$

则

$$
F'(D)=1-\mathrm{Ann} - \frac{(n+1)D^n}{n^nP}
= 1-\mathrm{Ann} - \frac{(n+1)D_P}{D}.
$$

代回牛顿步并化简（分子分母同乘 $D$）得到**闭式更新**：

$$
\boxed{
\; D_{\text{new}}
= \frac{\big(\mathrm{Ann}\,S + n\,D_P\big)\,D}
{(\mathrm{Ann}-1)\,D + (n+1)\,D_P}\;
}
\qquad(n=2).
$$

> 这就是代码中 `compute_d` 里那一行分式的来源。用闭式更新避免了“先求值再相减”的数值不稳定。

### 3. 如何计算 $D_P$（避免溢出）

直接算 $D^{n+1}$ 会放大溢出风险。实现中改为**迭代构造**：

$$
\begin{aligned}
& D_P \leftarrow D,\\
& D_P \leftarrow \frac{D_P\cdot D}{x_0\cdot n},\\
& D_P \leftarrow \frac{D_P\cdot D}{x_1\cdot n}.
\end{aligned}
$$

每一步都用安全的整数算术 `mul_div`（先 `checked_mul` 再 `checked_div`），这和库代码一致。

### 4. 初值、收敛与终止

* **初值**：取 $D_0=S=x_0+x_1$。它数量级正确且经验上 5–10 步内收敛。
* **收敛性直觉**：对 $n=2$，

  $$
  F'(D)=1-\mathrm{Ann}-\frac{3D^2}{4P}<0,\quad
  F''(D)=-\frac{3D}{2P}<0,
  $$

  $F$ 在 $D>0$ 上**严格递减且凹**，根唯一。以 $D_0=S$ 往往位于根左侧，牛顿迭代会单调地向根推进（实践中非常稳健）。
* **终止**：整数域中以 $|D_{k+1}-D_k|\le 1$ 作为停止条件；再加一个最大迭代数上限（代码中 `MAX_ITERS=256`）以防极端状态。

> 小结：`compute_d` 正是以上推导：用闭式牛顿步更新 $D$，用迭代方式安全计算 $D_P$，以 $|\Delta D|\le1$ 作为收敛判据。

---

## 二、在保持 $D$ 不变时求 $y$ 的牛顿法（`get_y`）

场景：你给币种 $i$ 注入（**费后**）$\Delta x_{\text{net}}$ 之后，得到新的 $x_i'=x_i+\Delta x_{\text{net}}$。此时保持上一步求出的不变量 $D$ 不变，解出**输出侧**的新余额 $y=x'_j$（从而输出量 $\Delta y = x_j - y$）。

### 1. 从不变量到标量方程

两币时有

$$
\mathrm{Ann}(x_i'+y) + D \;=\; \mathrm{Ann}D + \frac{D^{3}}{4\,x_i' y}.
$$

移项并乘以 $y$ 得到一个二次式：

$$
\mathrm{Ann}\,y^2 + \big(\mathrm{Ann}x_i' + D - \mathrm{Ann}D\big)\,y \;-\; \frac{D^3}{4x_i'} \;=\; 0.
$$

两边同除以 $\mathrm{Ann}$，记

$$
b \;\equiv\; x_i' + \frac{D}{\mathrm{Ann}}, 
\qquad
c \;\equiv\; \frac{D^3}{4\,\mathrm{Ann}\,x_i'},
$$

就得到**标准二次式**（这也是代码注释里写的形式）：

$$
\boxed{\; f(y)\;=\; y^2 + (b-D)\,y - c \;=\;0\;}.
$$

> 解释：在一般 $n>2$ 的实现中，$b,c$ 还会含有“除 $j$ 外所有币的乘积/和”；两币时它就化到了 $x_i'$ 上，写法最简。

### 2. 牛顿法推导与闭式更新

对

$$
f(y)\;=\; y^2 + (b-D)\,y - c
$$

求导：

$$
f'(y)=2y+(b-D).
$$

牛顿法：

$$
y_{\text{new}} \;=\; y - \frac{f(y)}{f'(y)}
\;=\; y - \frac{y^2+(b-D)y-c}{2y+b-D}.
$$

把分子分母整理一下，会出现一个非常简洁的**闭式迭代**：

$$
\boxed{\;
y \leftarrow \frac{y^2 + c}{2y + b - D}
\;}
$$

这正是代码中的

```text
y = (y*y + c) / (2*y + b - D)
```

——它等价于牛顿步，优点是只含“加、乘、除”，无减法差值，数值稳定。

### 3. 初值、正性与收敛

* **初值**：取 $y_0=D$（实现里就是 `y=d`）。
* **分母正性**：$\;2y + b - D \ge b + y > 0$（因 $y>0$、$b=x_i'+D/\mathrm{Ann}>0$），故不会除以 0。
* **收敛性直觉**：$f''(y)=2>0$，二次多项式**凸**，且在 $y>0$ 上只有一个正根；用牛顿法从一个合理的正初值出发会快速收敛（实践中 5–10 步）。
* **终止**：同 $D$，以 $|y_{k+1}-y_k|\le 1$ 作为停止条件（整数域）并设置迭代上限。

### 4. 为何不直接用求根公式？

二次方程当然可以用求根公式

$$
y=\frac{D-b\pm\sqrt{(b-D)^2+4c}}{2},
$$

但：

* 我们在**整数域**计算；出现开方、浮点会带来额外的舍入/实现复杂度；
* 牛顿形式只需加乘除，且能与 $n>2$ 的统一推导保持一致；
* 采用牛顿闭式步避免了“先算 $f$ 再相减”的灾难性消去误差，且每步都能用 `mul_div` 做安全有界的乘除。

### 5. 输出量与“减 1”策略

解出 $y$ 后，输出量

$$
\Delta y = x_j - y.
$$

为了守恒 $D$ 并避免因为整数商向下取整造成“多付出 1 个最小单位”，实现里再**保守减 1**：

$$
\Delta y_{\text{final}} = x_j - y \;-\; 1.
$$

这在链上实现里很常见（**保护 LP** 的保守出币策略）。

---

## 三、数值实现与健壮性要点（两种牛顿法共通）

1. **全程整数**：API 用 `u64` 表示 6 位小数的金额；内部全部提升到 `u128` 计算；分式通过 `mul_div(a,b,den)` 执行“乘后整除”，并用 `checked_*` 防溢出、分母判零。
2. **安全构造乘方/连分式**：像 $D_P$、$c$ 这种“高次/连乘”都**逐步**用 `mul_div` 构造，避免临时溢出。
3. **停止条件**：$|\Delta|\le 1$（整数域的“收敛半径”），再叠加 `MAX_ITERS` 保险丝。
4. **初值选择**：$D_0=S$ 与 $y_0=D$ 都有正确的量级且实践上很稳。
5. **错误处理**：索引不合法、零交易、零流动性、除零/溢出、未收敛都会早返回错误。
6. **为何效果好**：

   * $D$-方程的 $F(D)$ 在 $D>0$ 上单调递减、且只有唯一根；
   * $y$-方程是严格凸的一元二次，正根唯一；
     这两点保证了牛顿法从自然初值能**快速**且**稳定**地逼近解。

---

## 四、把推导和代码一一对应（速查表）

* **`compute_d`**

  * 目标：解 $F(D)=0$；
  * 核心：闭式牛顿步

    $$
    D \leftarrow \frac{(\mathrm{Ann}S+nD_P)\,D}{(\mathrm{Ann}-1)D+(n+1)D_P}
    $$
  * $D_P$ 用迭代 `mul_div` 构造；
  * 初值 `d=s`；停止：$|\Delta D|\le1$ 或迭代上限。

* **`get_y`**

  * 目标：在固定 $D$ 下解 $f(y)=0$，其中

    $$
    f(y)=y^2+(b-D)y-c,\quad
    b=x_i'+D/\mathrm{Ann},\;\ c=D^3/(4\,\mathrm{Ann}\,x_i');
    $$
  * 闭式牛顿步

    $$
    y \leftarrow \frac{y^2+c}{2y+b-D};
    $$
  * 初值 `y=d`；停止：$|\Delta y|\le1$ 或迭代上限；
  * 输出量 `dy = x_j - y - 1`（保守减 1）。

---

如果你愿意，我可以把上述推导**直接整理进你仓库的 README** 的「Mathematical Derivation」小节，并在公式下方补上与代码行号的对照标注，方便 code review 与教学使用。
